# Data Preprocessing - Neater Implementation
___

In [4]:
# Enable Intellisense
%config IPCompleter.greedy=True

import pandas as pd
import numpy as np
import io
import requests
import seaborn as sns
from matplotlib import pyplot, dates
import plotly.express as px 
import datetime
import plotly.graph_objects as go
import plotly.io as pio
sns.set()

# Annoying warning
pd.options.mode.chained_assignment = None  # default='warn'

In [5]:
# ['EC', 'FS', 'GP', 'KZN', 'LP', 'MP', 'NC', 'NW', 'WC']

## Gen Helper Functions

In [6]:
def df_from_url(df_url, pd_kwargs={}):
    df_req = requests.get(df_url).content
    df = pd.read_csv(io.StringIO(df_req.decode('utf-8')), **pd_kwargs)
    return df

In [7]:
tmp_kwargs = {"usecols":['date','total']}
df_from_url("https://raw.githubusercontent.com/dsfsi/covid19za/master/data/covid19za_provincial_cumulative_timeline_confirmed.csv", 
            tmp_kwargs).head()

,date,total
0,05-03-2020,1
1,07-03-2020,2
2,08-03-2020,3
3,09-03-2020,7
4,11-03-2020,13


# Cumulative/Daily Totals Per Day
Currently only applicable for confirmed cases and deaths.

In [8]:
def get_cum_daily(data_url, cum_col='total', index_col='date'): # kwargs={}):
#     data_req = requests.get(data_url).content
#     data = pd.read_csv(io.StringIO(data_req.decode('utf-8')), usecols=cols, index_col=['date'])
    cols = ['date', 'total']
    pd_kwargs = {"usecols":[cum_col, index_col],"index_col":[index_col]}
#     if usecols != []:
#         pd_kwargs.update({"usecols":usecols})
#     pd_kwargs.update(kwargs)
    data = df_from_url(data_url, pd_kwargs)
    data.reset_index(inplace=True)
    data['date'] = pd.to_datetime(data['date'], format='%d-%m-%Y')
    data.set_index('date', inplace = True)
    data.rename({cum_col:"cum_no"}, axis=1, inplace = True)
    data.ffill(inplace=True)
    
    data['daily_no'] = data['cum_no']
    # slice is required as first entry of diff will be NaN but data we want should be equal to the 
    # starting value
    data['daily_no'][1:] = data['cum_no'].diff()[1:]
    # Cast columns to integer
    data = data.astype('int32')
    return data

## Confirmed Cases

In [9]:
confirmed_cases_url = "https://raw.githubusercontent.com/dsfsi/covid19za/master/data/covid19za_provincial_cumulative_timeline_confirmed.csv"
confirmed_data = get_cum_daily(confirmed_cases_url)
confirmed_data

,cum_no,daily_no
date,,
2020-03-05,1,1
2020-03-07,2,1
2020-03-08,3,1
2020-03-09,7,4
2020-03-11,13,6
...,...,...
2020-05-02,6336,385
2020-05-03,6783,447
2020-05-04,7220,437


## Deaths

In [10]:
deaths_url = "https://raw.githubusercontent.com/dsfsi/covid19za/master/data/covid19za_provincial_cumulative_timeline_deaths.csv"
deaths_data = get_cum_daily(deaths_url)
deaths_data

,cum_no,daily_no
date,,
2020-03-27,1,1
2020-03-28,2,1
2020-03-30,3,1
2020-03-31,5,2
2020-04-03,9,4
2020-04-05,11,2
2020-04-06,12,1
2020-04-07,13,1
2020-04-08,18,5


## Tests

In [11]:
tests_url = "https://raw.githubusercontent.com/dsfsi/covid19za/master/data/covid19za_timeline_testing.csv"
tests_data = get_cum_daily(tests_url, 'cumulative_tests', 'date')
tests_data

,cum_no,daily_no
date,,
2020-02-11,61,61
2020-02-13,67,6
2020-02-14,71,4
2020-02-19,95,24
2020-02-20,106,11
...,...,...
2020-05-02,230686,13164
2020-05-03,245747,15061
2020-05-04,257541,11794


## Recoveries

In [12]:
tests_url = "https://raw.githubusercontent.com/dsfsi/covid19za/master/data/covid19za_timeline_testing.csv"
recovered_data = get_cum_daily(tests_url, 'recovered', 'date')
recovered_data

,cum_no,daily_no
date,,
2020-02-11,0,0
2020-02-13,0,0
2020-02-14,0,0
2020-02-19,0,0
2020-02-20,0,0
...,...,...
2020-05-02,2549,167
2020-05-03,2549,0
2020-05-04,2746,197


## Active Cases

In [13]:
active_data = confirmed_data[['cum_no']].copy().rename({"cum_no":"confirmed"}, axis = 1)
active_data = pd.concat([active_data, 
                         recovered_data[['cum_no']].copy().rename({"cum_no":"recovered"}, axis = 1)], 
                        axis =1)
# active_data.fillna(0, inplace=True)
active_data = active_data.iloc[9:]
active_data = active_data.ffill().fillna(0)
active_data

,confirmed,recovered
date,,
2020-03-05,1.0,0.0
2020-03-06,1.0,0.0
2020-03-07,2.0,0.0
2020-03-08,3.0,0.0
2020-03-09,7.0,0.0
...,...,...
2020-05-02,6336.0,2549.0
2020-05-03,6783.0,2549.0
2020-05-04,7220.0,2746.0


In [14]:
active_data['cum_no'] = active_data['confirmed'] - active_data['recovered']
active_data.drop(['confirmed','recovered'], axis=1, inplace=True)
active_data = active_data.astype('int32')
active_data

,cum_no
date,
2020-03-05,1
2020-03-06,1
2020-03-07,2
2020-03-08,3
2020-03-09,7
...,...
2020-05-02,3787
2020-05-03,4234
2020-05-04,4474


## All Cumulative/Totals Per Day

In [15]:
all_cum_data = confirmed_data[['cum_no']].rename({"cum_no":"confirmed"}, axis =1)
all_cum_data = pd.concat([
    all_cum_data, 
    tests_data[['cum_no']].rename({"cum_no":"tests"},axis=1),
    deaths_data[['cum_no']].rename({"cum_no":"deaths"},axis=1),
    recovered_data[['cum_no']].rename({"cum_no":"recovered"},axis=1),
    active_data[['cum_no']].rename({"cum_no":"active"},axis=1),

], axis=1)
# all_cum_data['recovered'] = recovered_data['cum_no']
# all_cum_data['active'] = active_data['cum_no']
all_cum_data.ffill(inplace=True)
all_cum_data.fillna(0, inplace=True)
all_cum_data = all_cum_data.astype('int32')
all_cum_data

,confirmed,tests,deaths,recovered,active
date,,,,,
2020-02-11,0,61,0,0,0
2020-02-13,0,67,0,0,0
2020-02-14,0,71,0,0,0
2020-02-19,0,95,0,0,0
2020-02-20,0,106,0,0,0
...,...,...,...,...,...
2020-05-02,6336,230686,123,2549,3787
2020-05-03,6783,245747,131,2549,4234
2020-05-04,7220,257541,138,2746,4474


## Derived Stats
Added to cumulative/totals per day `all_cum_data`
### Confirmed divided by Tests

In [16]:
all_cum_data['confirmed_div_by_tests'] = all_cum_data['confirmed']/all_cum_data['tests']
all_cum_data['confirmed_div_by_tests'] = all_cum_data['confirmed_div_by_tests'].round(3)
all_cum_data

,confirmed,tests,deaths,recovered,active,confirmed_div_by_tests
date,,,,,,
2020-02-11,0,61,0,0,0,0.000
2020-02-13,0,67,0,0,0,0.000
2020-02-14,0,71,0,0,0,0.000
2020-02-19,0,95,0,0,0,0.000
2020-02-20,0,106,0,0,0,0.000
...,...,...,...,...,...,...
2020-05-02,6336,230686,123,2549,3787,0.027
2020-05-03,6783,245747,131,2549,4234,0.028
2020-05-04,7220,257541,138,2746,4474,0.028


### Deaths divided by confirmed

In [17]:
all_cum_data['deaths_div_by_confirmed'] = all_cum_data['deaths']/all_cum_data['confirmed']
all_cum_data['deaths_div_by_confirmed'] = all_cum_data['deaths_div_by_confirmed'].round(3)
all_cum_data.fillna(0.000, inplace=True)
all_cum_data

,confirmed,tests,deaths,recovered,active,confirmed_div_by_tests,deaths_div_by_confirmed
date,,,,,,,
2020-02-11,0,61,0,0,0,0.000,0.000
2020-02-13,0,67,0,0,0,0.000,0.000
2020-02-14,0,71,0,0,0,0.000,0.000
2020-02-19,0,95,0,0,0,0.000,0.000
2020-02-20,0,106,0,0,0,0.000,0.000
...,...,...,...,...,...,...,...
2020-05-02,6336,230686,123,2549,3787,0.027,0.019
2020-05-03,6783,245747,131,2549,4234,0.028,0.019
2020-05-04,7220,257541,138,2746,4474,0.028,0.019


### Recovered divided by confirmed

In [18]:
all_cum_data['recovered_div_by_confirmed'] = all_cum_data['recovered']/all_cum_data['confirmed']
all_cum_data['recovered_div_by_confirmed'] = all_cum_data['recovered_div_by_confirmed'].round(3)
all_cum_data.fillna(0.000, inplace=True)
all_cum_data

,confirmed,tests,deaths,recovered,active,confirmed_div_by_tests,deaths_div_by_confirmed,recovered_div_by_confirmed
date,,,,,,,,
2020-02-11,0,61,0,0,0,0.000,0.000,0.000
2020-02-13,0,67,0,0,0,0.000,0.000,0.000
2020-02-14,0,71,0,0,0,0.000,0.000,0.000
2020-02-19,0,95,0,0,0,0.000,0.000,0.000
2020-02-20,0,106,0,0,0,0.000,0.000,0.000
...,...,...,...,...,...,...,...,...
2020-05-02,6336,230686,123,2549,3787,0.027,0.019,0.402
2020-05-03,6783,245747,131,2549,4234,0.028,0.019,0.376
2020-05-04,7220,257541,138,2746,4474,0.028,0.019,0.380


### Stats Per Million Population
[Source for stats](https://worldpopulationreview.com/countries/south-africa-population/)

In [19]:
sa_tot_population = 59195720
# total population rounded in millions
sa_tot_pop_mil = sa_tot_population/1000000
sa_tot_pop_mil

59.19572

In [20]:
all_cum_data['confirmed_per_mil'] = all_cum_data['confirmed']/sa_tot_pop_mil
all_cum_data['tests_per_mil'] = all_cum_data['tests']/sa_tot_pop_mil
all_cum_data['deaths_per_mil'] = all_cum_data['deaths']/sa_tot_pop_mil
all_cum_data['recovered_per_mil'] = all_cum_data['recovered']/sa_tot_pop_mil
all_cum_data['active_per_mil'] = all_cum_data['active']/sa_tot_pop_mil
tmp_cols = ['confirmed_per_mil','tests_per_mil','deaths_per_mil','recovered_per_mil','active_per_mil']
all_cum_data[tmp_cols] = all_cum_data[tmp_cols].round(2)
all_cum_data.fillna(0.00, inplace=True)
all_cum_data

,confirmed,tests,deaths,recovered,active,confirmed_div_by_tests,deaths_div_by_confirmed,recovered_div_by_confirmed,confirmed_per_mil,tests_per_mil,deaths_per_mil,recovered_per_mil,active_per_mil
date,,,,,,,,,,,,,
2020-02-11,0,61,0,0,0,0.000,0.000,0.000,0.00,1.03,0.00,0.00,0.00
2020-02-13,0,67,0,0,0,0.000,0.000,0.000,0.00,1.13,0.00,0.00,0.00
2020-02-14,0,71,0,0,0,0.000,0.000,0.000,0.00,1.20,0.00,0.00,0.00
2020-02-19,0,95,0,0,0,0.000,0.000,0.000,0.00,1.60,0.00,0.00,0.00
2020-02-20,0,106,0,0,0,0.000,0.000,0.000,0.00,1.79,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-05-02,6336,230686,123,2549,3787,0.027,0.019,0.402,107.03,3897.00,2.08,43.06,63.97
2020-05-03,6783,245747,131,2549,4234,0.028,0.019,0.376,114.59,4151.43,2.21,43.06,71.53
2020-05-04,7220,257541,138,2746,4474,0.028,0.019,0.380,121.97,4350.67,2.33,46.39,75.58


**Save to csv**

In [21]:
all_cum_data.to_csv('data/all_cum_data.csv')

### Change format of date

In [22]:
all_cum_data_alt = all_cum_data.copy().reset_index()


# Daily/Cumulative Data Per Prov Per Day

In [54]:
def get_cum_daily_by_prov(data_url):
    cols = ['date','EC', 'FS', 'GP', 'KZN', 'LP', 'MP', 'NC', 'NW','WC', 'UNKNOWN']
    pd_kwargs = {"usecols":cols}
    cum_data = df_from_url(data_url, pd_kwargs)
    cum_data.dropna(inplace=True)
    cum_data['date'] = pd.to_datetime(cum_data['date'], format='%d-%m-%Y')
    
    daily_data = cum_data.copy()
    daily_data_melt = daily_data.melt(id_vars=['date'], var_name='province', value_name='daily_no')
    daily_data_melt.set_index(['date'], inplace=True)
    
    
    cum_data_melt = cum_data.melt(id_vars=['date'], var_name='province', value_name='cum_no')
    cum_data_melt.set_index(['date'], inplace=True)
    
    data = pd.concat([cum_data_melt, daily_data_melt[['daily_no']]], axis = 1)
    return data

## Confirmed Cases Per Prov
Daily change and cumulative

In [60]:
confirmed_by_prov_timeline = get_cum_daily_by_prov("https://raw.githubusercontent.com/dsfsi/covid19za/master/" +
                      "data/covid19za_provincial_cumulative_timeline_confirmed.csv")
confirmed_by_prov_timeline

,province,cum_no,daily_no
date,,,
2020-03-05,EC,0.0,0.0
2020-03-07,EC,0.0,0.0
2020-03-08,EC,0.0,0.0
2020-03-09,EC,0.0,0.0
2020-03-11,EC,0.0,0.0
...,...,...,...
2020-05-02,UNKNOWN,0.0,0.0
2020-05-03,UNKNOWN,0.0,0.0
2020-05-04,UNKNOWN,0.0,0.0


**Save to csv**

In [62]:
confirmed_by_prov_timeline.to_csv("data/confirmed_by_prov_timeline.csv")

## Deaths Per Prov
Daily change and cumulative

In [64]:
deaths_by_prov_timeline = get_cum_daily_by_prov("https://raw.githubusercontent.com/dsfsi/covid19za/master/" +
                      "data/covid19za_provincial_cumulative_timeline_deaths.csv")
deaths_by_prov_timeline

,province,cum_no,daily_no
date,,,
2020-03-27,EC,0,0
2020-03-28,EC,0,0
2020-03-30,EC,0,0
2020-03-31,EC,0,0
2020-04-03,EC,0,0
...,...,...,...
2020-05-02,UNKNOWN,0,0
2020-05-03,UNKNOWN,0,0
2020-05-04,UNKNOWN,0,0


**Save to csv**

In [65]:
deaths_by_prov_timeline.to_csv("data/deaths_by_prov_timeline.csv")